In [2]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
InteractiveShell.ast_node_interactivity = "all"

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timezone

In [5]:
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_seq_items', 500)
pd.set_option("display.max_colwidth", None)

# Selection

In [6]:
df_ss = pd.read_excel("data/stock_stats.xlsx", sheet_name="stock_stats")

In [7]:
print("Total stocks: ", len(df_ss))
print("Total columns ", len(df_ss.columns))
df_ss.columns

Total stocks:  16887
Total columns  149


Index(['52WeekChange', 'SandP52WeekChange', 'address1', 'algorithm', 'ask',
       'askSize', 'averageDailyVolume10Day', 'averageVolume',
       'averageVolume10days', 'beta', 'bid', 'bidSize', 'bookValue',
       'category', 'city', 'coinMarketCapLink', 'companyOfficers', 'country',
       'currency', 'currencySymbol', 'currentPrice', 'currentRatio',
       'dateShortInterest', 'dayHigh', 'dayLow', 'debtToEquity',
       'dividendDate', 'dividendRate', 'dividendYield', 'earnings',
       'earningsGrowth', 'earningsQuarterlyGrowth', 'ebitda', 'ebitdaMargins',
       'enterpriseToEbitda', 'enterpriseToRevenue', 'enterpriseValue',
       'exDividendDate', 'exchange', 'exchangeDataDelayedBy', 'exchangeName',
       'fax', 'fiftyDayAverage', 'fiftyTwoWeekHigh', 'fiftyTwoWeekLow',
       'financialCurrency', 'firstTradeDateEpochUtc',
       'fiveYearAvgDividendYield', 'floatShares', 'forwardEps', 'forwardPE',
       'freeCashflow', 'fromCurrency', 'fullTimeEmployees', 'fundFamily',
       '

In [8]:
df_ss[df_ss["symbol"].isin(["IVR", "OXLC", "ORC", "TWO", "FRO"])].head(10).T

,5947,8133,11233,11360,15416
52WeekChange,0.726107,-0.39375,-0.326351,-0.194099,-0.421027
SandP52WeekChange,0.02896,0.02896,0.02896,0.02896,0.0116
address1,Iris House,Two Peachtree Pointe,3305 Flamingo Drive,Eight Sound Shore Drive,1601 Utica Avenue South
algorithm,NaN,NaN,NaN,NaN,NaN
ask,14.92,10.77,10.17,5.22,12.08
askSize,1300.0,1400.0,1300.0,1800.0,800.0
averageDailyVolume10Day,2493310.0,898890.0,741020.0,736410.0,1552910.0
averageVolume,2848738.0,1042613.0,718170.0,934983.0,1201468.0
averageVolume10days,2493310.0,898890.0,741020.0,736410.0,1552910.0
beta,0.254046,NaN,1.642505,1.131245,1.752638


In [9]:
df_ss["sector"].unique()

array(['Healthcare', 'Basic Materials', 'Financial Services',
       'Technology', 'Consumer Defensive', 'Industrials', 'Real Estate',
       'Consumer Cyclical', 'Communication Services', nan, 'Energy',
       'Utilities', 'Services', 'Consumer Goods', 'Financial',
       'Industrial Goods', 'Conglomerates'], dtype=object)

In [10]:
current_year = datetime.now().year

def from_epoch_time(value) -> datetime:
    return pd.to_datetime(value, unit="s")

df_ss.rename(columns=str.lower, inplace=True)
df_ss["lastdividenddate"] = df_ss["lastdividenddate"].apply(from_epoch_time)

INFO_FIELDS = [ "symbol", 
                "shortname", 
                "beta",
                "currentprice",
                "debttoequity", 
                "dividendrate", 
                "dividendyield", 
                "earnings",
                "exdividenddate",
                "fiveyearavgdividendyield",
                "forwardpe",
                "freecashflow",
                "lastdividenddate", 
                "lastdividendvalue",
                "pegratio",
                "pricetobook",
                "returnonequity",
                "sector",
                "trailingannualdividendyield",
                "trailingpe",
              ]

In [11]:
from scipy.optimize import minimize

# Read the data from the list of stocks
stocks = df_ss.copy()

# Replace infinity with NaN
stocks.replace([np.inf, -np.inf], np.nan, inplace=True)

In [12]:
# -- Five year dividend yield > median x 1.5 by sector
CRITERIA_FLD_HIST_DIV_YIELD_SECTOR = "criteria_hist_div_yield_sector"

def check_hist_div_yield_sector(df):
    return df['sector'].map(df.groupby('sector')['fiveyearavgdividendyield'].median()) * 1.2

criteria_hist_div_yield_sector = lambda df:  (df['fiveyearavgdividendyield'] > df[CRITERIA_FLD_HIST_DIV_YIELD_SECTOR])

# -- Five year dividend yield > minimum expected
CRITERIA_FLD_HIST_DIV_YIELD_MIN = "criteria_hist_div_yield_min"

def check_hist_div_yield_min():
    return 13

criteria_hist_div_yield_min = lambda df: df['fiveyearavgdividendyield'] > df[CRITERIA_FLD_HIST_DIV_YIELD_MIN]

# -- Dividend yield > minimum expected
CRITERIA_FLD_DIV_YIELD_MIN = "criteria_div_yield_min"

def check_div_yield_min():
    return 0.13

criteria_div_yield_min = lambda df: df['dividendyield'] > df[CRITERIA_FLD_DIV_YIELD_MIN]

# -- Pay dividend in the last year
CRITERIA_FLD_DIV_YEAR = "criteria_div_year"

def check_div_year(df):
    return  df['lastdividenddate'].dt.year.astype("Int64", errors="ignore")

criteria_div_year = lambda df:  df['criteria_div_year'] >= (current_year - 1)

# -- Sector is not blank
CRITERIA_FLD_SECTOR = "criteria_sector"

def check_sector(df):
    return ~df['sector'].isnull()

criteria_sector = lambda df: df[CRITERIA_FLD_SECTOR]

# -- Trailing PE
CRITERIA_FLD_TRAILING_PE = "criteria_trailing_pe"

def check_trailing_pe(df):
    return  df['sector'].map(df.groupby('sector')['trailingpe'].median()) * 1.0

criteria_trailing_pe = lambda df: df['trailingpe'] < df[CRITERIA_FLD_TRAILING_PE]


# Forward PE

CRITERIA_FLD_FWD_PE = "criteria_fwd_pe"

def check_fwd_pe(df):
    return  df['sector'].map(df.groupby('sector')['forwardpe'].median()) * 1.0

criteria_fwd_pe = lambda df: df['forwardpe'] < df[CRITERIA_FLD_FWD_PE]


# Return on Equity

CRITERIA_FLD_RETURN_ON_EQUITY = "criteria_return_on_equity"

def check_return_on_equity(df):
    return  df['sector'].map(df.groupby('sector')['returnonequity'].median()) * 1.0

criteria_return_on_equity = lambda df: df['returnonequity'] < df[CRITERIA_FLD_RETURN_ON_EQUITY]


# -- P/B ratio
CRITERIA_FLD_PB = "criteria_pb"

def check_pb(df):
    return  df['sector'].map(df.groupby('sector')['pricetobook'].median()) * 1.0

criteria_pb = lambda df: df['pricetobook'] < df[CRITERIA_FLD_PB]

# -- Debt to equity ratio
CRITERIA_FLD_DE = "criteria_de"

def check_de(df):
    return  df['sector'].map(df.groupby('sector')['debttoequity'].median()) * 1.0

criteria_de = lambda df: df['debttoequity'] < df[CRITERIA_FLD_DE]

criteria = {
    #CRITERIA_FLD_HIST_DIV_YIELD_SECTOR: criteria_hist_div_yield_sector,
    #CRITERIA_FLD_HIST_DIV_YIELD_MIN: criteria_hist_div_yield_min,
    CRITERIA_FLD_SECTOR: criteria_sector,
    CRITERIA_FLD_DIV_YEAR: criteria_div_year,
    CRITERIA_FLD_DIV_YIELD_MIN: criteria_div_yield_min,
    #CRITERIA_FLD_TRAILING_PE: criteria_trailing_pe,
    CRITERIA_FLD_FWD_PE: criteria_fwd_pe,
    #CRITERIA_FLD_PB: criteria_pb,
    #CRITERIA_FLD_DE: criteria_de,
    CRITERIA_FLD_RETURN_ON_EQUITY: criteria_return_on_equity,
}

CRITERIA_FIELDS = [
                   CRITERIA_FLD_HIST_DIV_YIELD_SECTOR,
                   CRITERIA_FLD_HIST_DIV_YIELD_MIN,
                   CRITERIA_FLD_DIV_YIELD_MIN,
                   CRITERIA_FLD_DIV_YEAR,
                   CRITERIA_FLD_SECTOR,
                   CRITERIA_FLD_FWD_PE,
                   CRITERIA_FLD_TRAILING_PE,
                   CRITERIA_FLD_PB,
                   CRITERIA_FLD_DE,
                   CRITERIA_FLD_RETURN_ON_EQUITY
                ]

stocks[CRITERIA_FLD_HIST_DIV_YIELD_SECTOR] = check_hist_div_yield_sector(stocks)
stocks[CRITERIA_FLD_HIST_DIV_YIELD_MIN] = check_hist_div_yield_min()
stocks[CRITERIA_FLD_DIV_YIELD_MIN] = check_div_yield_min()
stocks[CRITERIA_FLD_DIV_YEAR] = check_div_year(stocks)
stocks[CRITERIA_FLD_SECTOR] = check_sector(stocks)
stocks[CRITERIA_FLD_TRAILING_PE] = check_trailing_pe(stocks)
stocks[CRITERIA_FLD_FWD_PE] = check_fwd_pe(stocks)
stocks[CRITERIA_FLD_PB] = check_pb(stocks)
stocks[CRITERIA_FLD_DE] = check_de(stocks)
stocks[CRITERIA_FLD_RETURN_ON_EQUITY] = check_return_on_equity(stocks)

value_stocks = stocks.copy()
for name in criteria.keys():
    value_stocks = value_stocks.loc[criteria[name]]

print("Number of selected stocks - ", len(value_stocks))
value_stocks[CRITERIA_FIELDS + INFO_FIELDS].head(300).T

Number of selected stocks -  31


,196,443,1135,1763,1925,2365,3212,3881,4736,4737,4820,5294,7310,7600,8293,8652,9572,10390,10543,10687,10971,11016,11199,11360,11896,12225,13063,13725,14132,15263,15416
criteria_hist_div_yield_sector,6.45,6.45,6.45,6.45,3.0,6.45,6.45,4.908,4.908,4.908,6.45,3.084,6.45,4.908,2.916,6.45,6.45,4.908,6.45,6.45,6.45,4.908,6.45,4.908,6.45,4.908,6.45,6.45,6.45,6.45,6.45
criteria_hist_div_yield_min,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
criteria_div_yield_min,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13
criteria_div_year,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2022,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023
criteria_sector,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
criteria_fwd_pe,11.818258,11.818258,11.818258,11.818258,13.174074,11.818258,11.818258,7.530215,7.530215,7.530215,11.818258,9.326942,11.818258,7.530215,9.1,11.818258,11.818258,7.530215,11.818258,11.818258,11.818258,7.530215,11.818258,7.530215,11.818258,7.530215,11.818258,11.818258,11.818258,11.818258,11.818258
criteria_trailing_pe,14.859231,14.859231,14.859231,14.859231,19.539849,14.859231,14.859231,10.750097,10.750097,10.750097,14.859231,10.036585,14.859231,10.750097,14.885417,14.859231,14.859231,10.750097,14.859231,14.859231,14.859231,10.750097,14.859231,10.750097,14.859231,10.750097,14.859231,14.859231,14.859231,14.859231,14.859231
criteria_pb,0.778954,0.778954,0.778954,0.778954,1.090909,0.778954,0.778954,0.860432,0.860432,0.860432,0.778954,1.134752,0.778954,0.860432,1.234023,0.778954,0.778954,0.860432,0.778954,0.778954,0.778954,0.860432,0.778954,0.860432,0.778954,0.860432,0.778954,0.778954,0.778954,0.778954,0.778954
criteria_de,88.415,88.415,88.415,88.415,56.979,88.415,88.415,48.6685,48.6685,48.6685,88.415,21.6175,88.415,48.6685,71.7235,88.415,88.415,48.6685,88.415,88.415,88.415,48.6685,88.415,48.6685,88.415,48.6685,88.415,88.415,88.415,88.415,88.415
criteria_return_on_equity,0.03,0.03,0.03,0.03,0.088385,0.03,0.03,0.08642,0.08642,0.08642,0.03,-0.08498,0.03,0.08642,0.07314,0.03,0.03,0.08642,0.03,0.03,0.03,0.08642,0.03,0.08642,0.03,0.08642,0.03,0.03,0.03,0.03,0.03


In [13]:
# Research the shortlisted stocks
selected_symbols = ["ARLP", "ECC", "AGNC", "ARR", "BDN", "BIG", "CIM", "EFC", 
                    "HPP", "ICMB", "KREF", "MFA", "NLY", "NYMT", "OCCI", "OPI",
                    "PTMN", "RWT", "SLG", "TRTX", "TWO", "OXLC"
                   ]
selected_value_stocks = stocks[ stocks["symbol"].isin(selected_symbols)]
selected_value_stocks = selected_value_stocks[selected_value_stocks["fiveyearavgdividendyield"] > 10]
print(len(selected_value_stocks))
selected_value_stocks[CRITERIA_FIELDS + INFO_FIELDS].head(20).T

15


,443,1135,3212,4736,4737,4820,7600,9572,10543,10971,11199,11360,12225,15263,15416
criteria_hist_div_yield_sector,6.45,6.45,6.45,4.908,4.908,6.45,4.908,6.45,6.45,6.45,6.45,4.908,4.908,6.45,6.45
criteria_hist_div_yield_min,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
criteria_div_yield_min,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13
criteria_div_year,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023
criteria_sector,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
criteria_fwd_pe,11.818258,11.818258,11.818258,7.530215,7.530215,11.818258,7.530215,11.818258,11.818258,11.818258,11.818258,7.530215,7.530215,11.818258,11.818258
criteria_trailing_pe,14.859231,14.859231,14.859231,10.750097,10.750097,14.859231,10.750097,14.859231,14.859231,14.859231,14.859231,10.750097,10.750097,14.859231,14.859231
criteria_pb,0.778954,0.778954,0.778954,0.860432,0.860432,0.778954,0.860432,0.778954,0.778954,0.778954,0.778954,0.860432,0.860432,0.778954,0.778954
criteria_de,88.415,88.415,88.415,48.6685,48.6685,88.415,48.6685,88.415,88.415,88.415,88.415,48.6685,48.6685,88.415,88.415
criteria_return_on_equity,0.03,0.03,0.03,0.08642,0.08642,0.03,0.08642,0.03,0.03,0.03,0.03,0.08642,0.08642,0.03,0.03


In [14]:
def objective(weights, stocks):
    returns = np.array([stock['dividendyield'] for stock in stocks])
    cov_matrix = np.cov(returns)
    portfolio_return = np.dot(weights, returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return -portfolio_return / portfolio_volatility

In [15]:
def optimize_portfolio(stocks):
    sectors = set([stock['sector'] for stock in stocks])
    num_sectors = len(sectors)
    num_stocks = len(stocks)
    
    def sum_constraint(x):
        return np.sum(x) - 1.0
    
    constraints = [
        {'type': 'eq', 'fun': sum_constraint}
    ]
    
    bounds = [(0, 1) for i in range(num_stocks)]
    
    result = minimize(objective, np.ones(num_stocks) / num_stocks, args=(stocks,), method='SLSQP', bounds=bounds, constraints=constraints)
    
    return result.x

shortlisted_stocks = selected_value_stocks[INFO_FIELDS].to_dict("records")
portfolio_weights = optimize_portfolio(shortlisted_stocks)
counter = 0
print("Number of stocks - " , len(shortlisted_stocks))
for stock in shortlisted_stocks:
    print(f"{stock['symbol']} - {portfolio_weights[counter]}")
    counter = counter + 1

Number of stocks -  15
AGNC - 0.0636053323933296
ARR - 0.08046467216847399
CIM - 0.07775454061078514
ECC - 0.06108433134113401
ECC - 0.06108372620715817
EFC - 0.06058598931833054
ICMB - 0.06509380414012746
MFA - 0.05598343279160247
NLY - 0.05721451091474263
NYMT - 0.06790310943770418
OPI - 0.06641651972044134
OXLC - 0.07153944402769154
PTMN - 0.05528336989112594
TRTX - 0.07317778130379589
TWO - 0.08280943573356533


In [16]:
sectors = set([stock['sector'] for stock in shortlisted_stocks])
print(sectors)

{'Financial Services', 'Real Estate'}


In [17]:
AMT_INVEST = 1300
total_amt = 0
counter = 0
for stock in shortlisted_stocks:
    amt = portfolio_weights[counter] * AMT_INVEST
    total_amt = total_amt + amt
    counter = counter + 1
    print(f"{counter} - {stock['symbol']},{stock['shortname']} - {round(amt,2)}")
    #print(f"{stock['symbol']},{stock['shortname']}")

print("Total amt: ", total_amt)

1 - AGNC,AGNC Investment Corp. - 82.69
2 - ARR,ARMOUR Residential REIT, Inc. - 104.6
3 - CIM,Chimera Investment Corporation - 101.08
4 - ECC,Eagle Point Credit Company Inc. - 79.41
5 - ECC,Eagle Point Credit Company Inc. - 79.41
6 - EFC,Ellington Financial Inc. - 78.76
7 - ICMB,Investcorp Credit Management BD - 84.62
8 - MFA,MFA Financial, Inc. - 72.78
9 - NLY,Annaly Capital Management Inc. - 74.38
10 - NYMT,New York Mortgage Trust, Inc. - 88.27
11 - OPI,Office Properties Income Trust - 86.34
12 - OXLC,Oxford Lane Capital Corp. - 93.0
13 - PTMN,Portman Ridge Finance Corporati - 71.87
14 - TRTX,TPG RE Finance Trust, Inc. - 95.13
15 - TWO,Two Harbors Investment Corp - 107.65
Total amt:  1300.0000000000105


In [18]:
print(value_stocks['dividendyield'].sum())

5.533299990000001


Stocks to track

- GOGL

  
- BWLLF
- PBR-A
- PBR 
- ORC - monthly
- OXLC - monthly

OXLC
TWO
ECC
ARLP


## Piotroski Score

In [19]:
# TODO - https://python.plainenglish.io/finding-the-best-value-stock-with-piotroski-score-in-python-5a793580226b

## Enter, Exit and Stop Loss

In [20]:
# TODO -  Strategy to buy - current share price?